In [1]:

#using selenium for better extractions of the page as few pages were not loading 
!pip install selenium
#web driver will open a web browser to load the page
!pip install webdriver-manager

In [2]:
#retriving the links for movies
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
URL = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
imdb = "https://www.imdb.com"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
page = requests.get(URL,headers=headers)
soup = BeautifulSoup(page.content, "html.parser")
movies = soup.find_all('td', class_ = 'titleColumn')
movie_links=[]

for i in movies:
    #all the links are appended to a list for the 250 movies
    link = i.find("a")
    movie_link = link.get("href")
    movie_links.append(imdb+movie_link)


    

In [3]:
# Object contains all the information of a specific movie
class Movie:
    def __init__(self,title,director,genre):
        self.title = title
        self.director = director
        self.genre = genre



In [4]:
import time
import re
from selenium import webdriver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
n = 1
all_movies = []
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
try:
    for movie_link in movie_links:





#         request the page to load from the links provided on top
        driver.get(movie_link)
        content = driver.page_source
# once the content is then parsed using beautiful soup
        soup = BeautifulSoup(content, "html.parser")


        #title has been extracted from an H1 tag which unique has data attributes 

        title=soup.find("h1",attrs={'data-testid' : "hero-title-block__title"})


    #     it is then converted into lower case for easier finding
        title=title.get_text().lower()
    # the directors are extracted from the head of the html body in a meta tag
        meta = soup.find("meta",property="og:description")
        meta = meta["content"]
        dirc = re.search("",meta)
    #this will help in extracting all the directors
        y = re.findall("Directed by ([^\.]*)",meta)

        y = y[0].lower()
    # more than one directors will be split in to multiple elements in a list
        meta= y.split(",")
    # used map function to make sure there are no empty spaces around the directors
        directors = list(map(str.strip,meta))

        genre_row = soup.find('li', attrs = {'class' : 'ipc-metadata-list__item', 'data-testid' : 'storyline-genres'})
        genre_tags = genre_row.find_all('a')
        genres = []
    #this will iterate through all the a tags and extract their text and then append it in a list.
        for i in genre_tags:
            genre = i.get_text()
            genres.append(genre)


        movie = Movie(title, directors,genres)

        all_movies.append(movie)
        print(f"Movie {n} completed")
        n = n + 1

        time.sleep(0.5)
except:
    print("Page didn't load rerun this code")
    



====== WebDriver manager ======
Current google-chrome version is 119.0.6045
Get LATEST chromedriver version for 119.0.6045 google-chrome


ValueError: There is no such driver by url https://chromedriver.storage.googleapis.com/LATEST_RELEASE_119.0.6045

In [32]:
import csv

movie_file = open('Pranav_movies.csv', mode='w', newline='') #line 1
movie_writer = csv.writer(movie_file) #line 2
for i in all_movies:
#     since there are more than one directors in few movies they are separated by ; for better understanding.
     movie_writer.writerow([i.title,";".join(i.director), ";".join(i.genre)])
movie_file.close()



In [116]:

def movies_genres(all_movies):
    movies1 = [] #will append all the movies of that specific director to this list
    genres1 = [] #will append all the genres of that specific director to this list.

    name=input("Who do you want to check? ").lower()
    
    for i in all_movies:
        #if the director is present in the list it will provide the remaining information from its object
        if name in i.director:
            movies1.append(i.title.title())
            genres1.extend(i.genre)
#  if the director was not found that means the length will be zero which will return false and which is handled by the try block inside the main code
    if len(genres1)==0:
        return False
    counts={}
# keeping a count of all the genres and adding if there are more into the dictionary
    for g in genres1 :
# if the key does not exist it will give the value as zero if it exist it will increment the value by 1
        counts[g]=counts.get(g,0)+1
    
      
    return movies1,counts,name

In [117]:
# to provide all the genres in one line append the key and value together into a single string in list
def genres_count(counts):
    total_genres=[]
    for key,value in counts.items():
        total_genres.append(f"{key}: {value}")
        
    return total_genres



In [118]:
# to find the similar score this function will return the two list of values of the genres
def vector_distance(x,y):
    xset=set(x.keys())
    yset=set(y.keys())

    total = list(xset.union(yset))
    total.sort()


    for i in total:
        x[i]=x.get(i,0)
        y[i]=y.get(i,0)


    valueA=[]
    valueB=[]

    for j in total:
        a = x.get(j)
        b = y.get(j)
        
        valueA.append(a)
        valueB.append(b)
    return valueA,valueB






In [119]:
# cosine value 
def similarities(valueA,valueB):
    ab=0
    asqr=0
    bsqr=0
    for i in range(len(valueA)):
        ab = ab + valueA[i]*valueB[i]
        asqr = asqr + valueA[i]**2


        bsqr = bsqr + valueB[i]**2

    cosine = ab/((asqr**0.5)*(bsqr**0.5))
    return cosine

In [122]:
joinstr = ", " #to use to join the list of directors or genres.
while True:
    main_input = input("What do you want to check on IMDB? (Please choose ‘movie’, ‘director’, or ‘comparison or stop: ").lower()
    try:
        if main_input=="movie":
            option_input=input("What movie do you want to check? ").lower()
            trigger = 0 
            for i in all_movies:
                if option_input in i.title:
                    trigger=trigger+1
                    print(f"\n The director of this movie is/are {(joinstr.join(i.director)).title()}\n\n And the genres is/are {joinstr.join(i.genre)}")
#             this variable will trigger if the movie is not found in the list
            if trigger ==0:
                print("Invaild Movie")

        elif main_input=="director":
    #calling the function and storing the output of the function in three variables     
            try:
                movies1,count,name=movies_genres(all_movies)
                total_genres= genres_count(count)
# title function will capitalize the first alphabet for the name of the director.
                print(f"{name.title()}  has directed {joinstr.join(movies1)} ")
                print(f"And most often played genres are {joinstr.join(total_genres)} ")
            except:
                print("Director Not found")






        elif main_input=="comparison":

                try:
                    print("Director 1")
                    movie1,count1,name1=movies_genres(all_movies)

                    print("Director 2")
                    movie2,count2,name2=movies_genres(all_movies) 

                    total_genres1= genres_count(count1)
                    total_genres2= genres_count(count2)

                    vectordist1,vectordist2=vector_distance(count1,count2)

                    print(f"{name1.title()}  has directed {joinstr.join(movie1)} ")
                    print(f"And most often played genres are {joinstr.join(total_genres1)} ")
                    print(name2)
                    print(f"\n\n{name2.title()}  has directed {joinstr.join(movie2)} ")
                    print(f"And most often played genres are {joinstr.join(total_genres2)} ")


                    similar = similarities(vectordist1,vectordist2)
                    print("similarity score is ",similar)
                except:
                    print("Invalid input try again")

        elif main_input=="stop":
            break
        else:
            print("Enter a valid input")
            continue
    except:
        print("something went wrong ")

# quentin tarantino
# ridley scott

What do you want to check on IMDB? (Please choose ‘movie’, ‘director’, or ‘comparison or stop: movie
What movie do you want to check? nejfne
Invaild Movie
What do you want to check on IMDB? (Please choose ‘movie’, ‘director’, or ‘comparison or stop: director
Who do you want to check? jsnd
Director Not found
What do you want to check on IMDB? (Please choose ‘movie’, ‘director’, or ‘comparison or stop: director
Who do you want to check? quentin tarantino
Quentin Tarantino  has directed Pulp Fiction, Django Unchained, Inglourious Basterds, Reservoir Dogs, Kill Bill: Vol. 1 
And most often played genres are Crime: 3, Drama: 5, Western: 1, Adventure: 1, War: 1, Thriller: 2, Action: 1 
What do you want to check on IMDB? (Please choose ‘movie’, ‘director’, or ‘comparison or stop: comparison
Director 1
Who do you want to check? quentin tarantino
Director 2
Who do you want to check? ridley scott
Quentin Tarantino  has directed Pulp Fiction, Django Unchained, Inglourious Basterds, Reservoir Dogs